In [1]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline

In [2]:
os.environ['COBAYA_NOMPI'] = 'True'
from cobaya.yaml          import yaml_load_file
from cobaya.samplers.mcmc import plot_progress
#
from getdist.mcsamples    import MCSamplesFromCobaya
from getdist.mcsamples    import loadMCSamples
import getdist.plots      as     gdplt


from cobaya.model import get_model

sys.path.append('/global/cscratch1/sd/sfschen/BOSSxPlanck/')
sys.path.append('/global/cscratch1/sd/sfschen/BOSSxPlanck/emulator/')
sys.path.append('/global/cscratch1/sd/sfschen/BOSSxPlanck/rsd_likelihood/')
sys.path.append('/global/cscratch1/sd/sfschen/BOSSxPlanck/gxk_likelihood/')
#
import os

In [3]:
info = yaml_load_file("joint_z3_lcdm.yaml")
model = get_model(info)

[model] *WARNING* Ignored blocks/options: ['sampler', 'output', 'stop_on_error']
[parameterization] *WARNING* There are repeated parameter labels: {'SN^{N3}': ['SN_NGCz3', 'SN_SGCz3']}
Loading Taylor series.
['z3'] ['/global/cscratch1/sd/sfschen/boss_data_renorm/xi/z3.xi']
['NGCz3', 'SGCz3'] ['/global/cscratch1/sd/sfschen/boss_data_renorm/pk/pk_NGC_z3.dat', '/global/cscratch1/sd/sfschen/boss_data_renorm/pk/pk_SGC_z3.dat']
We are here!


In [107]:
pars = {'H0': 71.957642,
 'omegam': 0.30624422,
 'logA': 2.7559452999999996,
 'b1_NGCz3': 1.2879399,
 'b2_NGCz3': -1.6734879,
 'bs_NGCz3': 2.6070984,
 'alpha0_NGCz3': 27.3449,
 'alpha2_NGCz3': -10.436179,
 'SN0_NGCz3': -718.57285,
 'SN2_NGCz3': -2354.3215,
 'alpha_x_NGCz3': -1.5083223999999997,
 'smag_NGCz3': 0.91977681,
 'b1_SGCz3': 1.3055484,
 'b2_SGCz3': 0.30626086,
 'bs_SGCz3': 0.52783089,
 'alpha0_SGCz3': 14.346956,
 'alpha2_SGCz3': -4.709547799999999,
 'SN0_SGCz3': -1445.2167,
 'SN2_SGCz3': -55322.03599999999,
 'alpha_x_SGCz3': 6.3628728,
 'smag_SGCz3': 0.98791825,
 'B1_z3': 1.5349626,
 'F_z3': 1.0959386,
 'M0_z3': 0.0023621726000000003,
 'M1_z3': -0.11851115,
 'Q0_z3': -0.005235214,
 'Q1_z3': 0.7994913}

pars_list = pars.keys()

In [52]:
model.likelihood.keys()

dict_keys(['joint_boss_likelihoods_emu_lnA.FullShapePlusBAOEmu_Z3', 'gxk_likelihood.GxKLikelihood'])

In [5]:
model.logposterior(pars)
glik = model.likelihood['gxk_likelihood.GxKLikelihood']
rlik = model.likelihood['joint_boss_likelihoods_emu_lnA.FullShapePlusBAOEmu_Z3']

In [94]:
dPs = {}
Fishers = {}

for lik in model.likelihood.keys():
    dPs[lik] = {}
    Fishers[lik] = {}

In [108]:
err = 0.05

for lik in model.likelihood.keys():
    
    for param_str in pars_list:
        param = pars[param_str]
    
        pars_plus = pars.copy()
        pars_plus[param_str] = param + err
    
        pars_minus = pars.copy()
        pars_minus[param_str] = param - err

        model.logposterior(pars_plus)
        obs_plus = 1.0 * model.likelihood[lik].obs
        
        model.logposterior(pars_minus)
        obs_minus = 1.0 * model.likelihood[lik].obs
        
        dPs[lik][param_str] = (obs_plus - obs_minus)/(2*err)

In [109]:
for lik in model.likelihood.keys():
    
    print(lik)
    
    Fij = np.zeros( (len(pars.keys()),)*2 )
    
    for ii, param_ii in enumerate(pars_list):
        for jj, param_jj in enumerate(pars.keys()):
            
            Fij[ii,jj] = np.dot(dPs[lik][param_ii],\
                                np.dot(model.likelihood[lik].cinv,\
                                       dPs[lik][param_jj]))
            
    Fishers[lik] = np.array(Fij)
    
    
            

joint_boss_likelihoods_emu_lnA.FullShapePlusBAOEmu_Z3
gxk_likelihood.GxKLikelihood


In [110]:
Ftot = 0
for lik in model.likelihood.keys():
    print(lik)
    Ftot += Fishers[lik]

joint_boss_likelihoods_emu_lnA.FullShapePlusBAOEmu_Z3
gxk_likelihood.GxKLikelihood


In [113]:
np.diag(np.linalg.inv(Ftot))**0.5

array([1.32350622e+00, 1.78541875e-02, 3.21641270e-01, 6.98578011e-01,
       7.25991260e+00, 1.15716934e+01, 1.17107097e+02, 1.00425429e+02,
       8.54500136e+03, 5.40194123e+04, 3.43836873e+01, 3.13680145e+00,
       5.84205598e-01, 9.93373546e+00, 1.39075160e+01, 1.64847982e+02,
       1.70841772e+02, 1.76067068e+04, 7.72567100e+04, 3.79246271e+01,
       3.56417151e+00, 5.10028756e-01, 5.16761595e-01, 1.19367410e-03,
       1.41593884e-01, 2.19328346e-03, 2.32617599e-01])

In [117]:

np.savetxt(info['output'] + '.covini', np.linalg.inv(Ftot))